In [2]:
import re
import metrics
import importlib
import numpy as np
import pandas as pd
import configparser

from sqlalchemy import create_engine

In [3]:
config = configparser.ConfigParser()
config.read('config.ini')

POSTGRES_DB = config['postgresql']['database']
POSTGRES_USER = config['postgresql']['user']
POSTGRES_HOST = config['postgresql']['host']
POSTGRES_PORT = config['postgresql']['port']
POSTGRES_PASSWORD = config['postgresql']['password']

engine = create_engine(
    f'postgresql+psycopg2://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}')

In [6]:
# Load the DataFrames from the database
df_db_info = pd.read_sql("""SELECT * FROM aidm.db_info;""", engine)
df_schema_info = pd.read_sql("""SELECT * FROM aidm.schema_info""", engine)
df_table_info = pd.read_sql("""SELECT * FROM aidm.table_info""", engine)
df_feature_info = pd.read_sql("""SELECT * FROM aidm.feature_info""", engine)
df_metric_info = pd.read_sql("""SELECT * FROM aidm.metric_info;""", engine)
df_table_completeness = pd.read_sql("""SELECT * FROM aidm.table_completeness;""", engine)
df_table_uniqueness = pd.read_sql("""SELECT * FROM aidm.table_uniqueness;""", engine)
df_table_redundancy = pd.read_sql("""SELECT * FROM aidm.table_redundancy;""", engine)
df_table_integration = pd.read_sql("""SELECT * FROM aidm.table_pk_fk;""", engine) 
df_table_index = pd.read_sql("""SELECT * FROM aidm.table_index""", engine)
df_table_comment = pd.read_sql("""SELECT * FROM aidm.table_comment""", engine)

In [8]:
df_user = pd.read_sql("""SELECT * FROM hrsv_raw_dataset.users""", engine)
df_logbook = pd.read_sql("""SELECT * FROM hrsv_raw_dataset.users""", engine)
df_presence = pd.read_sql("""SELECT * FROM hrsv_raw_dataset.presences""", engine)
df_holiday = pd.read_sql("""SELECT * FROM hrsv_raw_dataset.holiday""", engine)
df_role = pd.read_sql("""SELECT * FROM hrsv_raw_dataset.roles""", engine)

In [9]:
df_logbook.head(3)

,_id,name,email,address,phone,image_url,workplace_id,team_id,company_role_id,system_role,fid,is_active,is_verified,is_synced,created_at,updated_at,hashed_password,updated_by
0,64c3838f6099f3a9d66042f6,Ryan Kurniawan,ryan.kurniawan@it.sharingvision.com,,,,76a9a8fc-2eb5-4743-907c-205b5e06323e,c597bcba-2cce-4686-976a-046f331ac01b,9c22f89c-4d39-4456-bdbb-6745ca97cbfc,User,0EuKYex9wJVkzWsAwqaavXliOEz2,True,True,True,2023-05-12 05:13:29,2023-06-19 02:02:03,$2b$12$gMZiraeAaDjV5udiV3MBmefZo7gOQq/57BjZ3z9...,None
1,64c3838f6099f3a9d66042f8,Herlinda Rosa,herlindarosa77@gmail.com,,,,OX4pqyuuI6muduc5ZbrF,766f85d7-0eb3-4686-8264-0c921c967c44,11188d24-aab0-4d46-93a4-8da4020e2d01,User,0bpDczexdNcl4VYNgJVEgMRyl9k2,True,True,True,2023-05-12 06:52:26,2023-05-12 11:55:54,$2b$12$T..ngsctvdn/VO8NCf//7.cTr5je66g6JR5qVTm...,aL2JQYe1Dqgemta29pXm6YOnAs52
2,64c3838f6099f3a9d66042f9,Stevefanus,stevefanus@lapi.my.id,,,,a6e20886-d9b9-4c1e-94d1-785acaa5492d,00736315-d91d-4835-af37-28c96ab99673,be36d1f3-afd1-474b-875d-62bf0c9734f4,User,0mTdXB1rN2ffb5wKQt4XO8ZoB3r2,True,True,False,2023-05-12 03:21:41,2023-06-20 09:07:45,$2b$12$OAGaZnmXts2ahZjhtE0Bs.73RKatpt80CjCtKuO...,aL2JQYe1Dqgemta29pXm6YOnAs52


### General Function

In [11]:
importlib.reload(metrics)

<module 'metrics' from 'C:\\Users\\Sony LOQ\\Data Science\\aidm\\Python Functions\\metrics.py'>

In [12]:
evaluator = metrics.DataQualityEvaluator(engine)
data_service = metrics.DataService(engine)
external_data_service = metrics.ExternalDatabaseService()

In [13]:
evaluator.table_list().head(5)

,id_db,database,id_schema,schema,id_table,table
0,1,db_data_catalog,2,public,1,datamart_count_attribute
1,1,db_data_catalog,3,raw_metadata,2,hrsv_metadata_1
2,1,db_data_catalog,3,raw_metadata,3,hrsv_metadata_2
3,1,db_data_catalog,3,raw_metadata,4,mariadb_metadata_1
4,1,db_data_catalog,3,raw_metadata,5,mariadb_metadata_2


### Completeness Function

In [68]:
evaluator.completeness_column(df_user["name"])

,row_total,row_null,row_complete,complete_perc
0,251,0,251,100.0


In [74]:
evaluator.completeness_table(df_user).head(5)

,column,row_total,row_null,row_complete,complete_perc
0,_id,251,0,251,100.0
0,name,251,0,251,100.0
0,email,251,0,251,100.0
0,address,251,0,251,100.0
0,phone,251,0,251,100.0


In [76]:
evaluator.completeness_table_all().head(5)

,id_db,database,id_schema,schema,id_table,table,column,row_total,row_null,row_complete,complete_perc
0,1,db_data_catalog,2,public,1,datamart_count_attribute,count_db,0,0,0,0.0
1,1,db_data_catalog,2,public,1,datamart_count_attribute,count_tables,0,0,0,0.0
2,1,db_data_catalog,2,public,1,datamart_count_attribute,count_row,0,0,0,0.0
3,1,db_data_catalog,3,raw_metadata,2,hrsv_metadata_1,table_catalog,173,0,173,100.0
4,1,db_data_catalog,3,raw_metadata,2,hrsv_metadata_1,table_schema,173,0,173,100.0


### Uniqueness Function

In [78]:
evaluator.uniqueness_column(df_logbook["name"])

,row_total,value_unique,unique_perc
0,251,251,100.0


In [84]:
evaluator.uniqueness_table(df_logbook).head(3)

,column,row_total,value_unique,unique_perc
0,_id,251,251,100.0
0,name,251,251,100.0
0,email,251,251,100.0


In [86]:
evaluator.uniqueness_table_all().head(3)

,id_db,database,id_schema,schema,id_table,table,column,row_total,value_unique,unique_perc
0,1,db_data_catalog,2,public,1,datamart_count_attribute,count_db,0,0,0.0
1,1,db_data_catalog,2,public,1,datamart_count_attribute,count_tables,0,0,0.0
2,1,db_data_catalog,2,public,1,datamart_count_attribute,count_row,0,0,0.0


### Validity Function

In [28]:
evaluator.validity_column(df_user["email"], "email").head(3)

,value,is_valid
0,ryan.kurniawan@it.sharingvision.com,True
1,herlindarosa77@gmail.com,True
2,stevefanus@lapi.my.id,True


### Redundancy Function

In [30]:
evaluator.redundancy_table(df_user)

,row_total,row_duplicates,redundancy_perc
0,79,0,0.0


### Timeliness Function

In [34]:
evaluator.timeliness_table(df_logbook, "updated_at")

,update_time_column,average_update_time
0,updated_at,5.96


### Integrity Function

In [40]:
evaluator.integrity_column(df_logbook, "_id", df_user, "_id")

,row_total,column_name,column_reference_name,value_integrous,integrity_perc
0,251,_id,_id,251,100.0


### Consistency Function

In [46]:
evaluator.consistency_column(df_logbook, "name", "_id", df_user, "name", "_id")

,row_total,column_name,column_reference_name,value_consistency,consistency_perc
0,251,name,name,251,100.0


### Mask Function -- belum akurat

In [73]:
evaluator.masking_column(df_logbook, "name")

,row_total,column,value_masked,masked_perc
0,251,name,1,0.4


In [78]:
df = pd.DataFrame({"id":["1XXXXX123","2XXXXX133","3XXXXX125","4XXXXX323","5XXXXX126","6XXXXX122","8XXXXX163","1XXXXX923","2XXXXX124"]})
evaluator.masking_column(df, "id")

,row_total,column,value_masked,masked_perc
0,9,id,9,100.0


### Primary Key Function

In [87]:
evaluator.primary_key_table().head(3)

,id_db,database,id_schema,schema,id_table,table,is_primary_key,primary_key
0,1,db_data_catalog,2,public,1,datamart_count_attribute,0,
1,1,db_data_catalog,3,raw_metadata,2,hrsv_metadata_1,0,
2,1,db_data_catalog,3,raw_metadata,3,hrsv_metadata_2,0,


In [89]:
evaluator.primary_key_schema().head(3)

,id_db,database,id_schema,schema,primary_key_perc
0,1,db_data_catalog,2,public,0.0
1,1,db_data_catalog,3,raw_metadata,0.0
2,1,db_data_catalog,4,datamart_layer_1,75.0


### Foreign Key Function

In [98]:
evaluator.foreign_key_table().head(3)

,id_db,database,id_schema,schema,id_table,table,foreign_key,is_foreign_key_valid,foreign_key_perc
0,1,db_data_catalog,2,public,1,datamart_count_attribute,{},{},0.0
1,1,db_data_catalog,3,raw_metadata,2,hrsv_metadata_1,{},{},0.0
2,1,db_data_catalog,3,raw_metadata,3,hrsv_metadata_2,{},{},0.0


In [100]:
evaluator.foreign_key_schema().head(3)

,id_db,database,id_schema,schema,foreign_key_perc
0,1,db_data_catalog,2,public,0.0
1,1,db_data_catalog,3,raw_metadata,0.0
2,1,db_data_catalog,4,datamart_layer_1,50.0


### Indexing Function

In [115]:
evaluator.index_table().head(3)

,id_db,database,id_schema,schema,id_table,table,is_indexed,index
0,1,db_data_catalog,2,public,1,datamart_count_attribute,0,None
1,1,db_data_catalog,3,raw_metadata,2,hrsv_metadata_1,0,None
2,1,db_data_catalog,3,raw_metadata,3,hrsv_metadata_2,0,None


In [117]:
evaluator.index_schema().head(3)

,id_db,database,id_schema,schema,index_perc
0,1,db_data_catalog,2,public,0.0
1,1,db_data_catalog,3,raw_metadata,33.0
2,1,db_data_catalog,4,datamart_layer_1,100.0


### Comment Function

In [121]:
evaluator.comment_table().head(3)

,id_db,database,id_schema,schema,id_table,table,is_commented,comment
0,1,db_data_catalog,2,public,1,datamart_count_attribute,0,None
1,1,db_data_catalog,3,raw_metadata,2,hrsv_metadata_1,0,None
2,1,db_data_catalog,3,raw_metadata,3,hrsv_metadata_2,0,None


In [123]:
evaluator.comment_schema().head(3)

,id_db,database,id_schema,schema,comment_perc
0,1,db_data_catalog,2,public,0.0
1,1,db_data_catalog,3,raw_metadata,0.0
2,1,db_data_catalog,4,datamart_layer_1,0.0


### Add Records

In [212]:
data_service.add_agent("Sharing Vision Indonesia", "SVI")

Agent 'Sharing Vision Indonesia' added successfully with ID 3.


3

In [216]:
data_service.add_database("Data Catalog", "PostgreSQL", id_agent=3, hostname="10.82.44.174", port=5432, instance="db_data_catalog", description="Katalog data HR SV")

Database 'Data Catalog added successfully with ID 5'


5

### Delete Records

In [199]:
data_service.delete_agent(1)

Agent with ID 1 deleted successfully.


In [210]:
data_service.delete_database(1)

Database with ID 1 deleted successfully.


### Collect and Add Records

In [467]:
test_db = config['test']['database']
test_user = config['test']['user']
test_host = config['test']['host']
test_port = config['test']['port']
test_password = config['test']['password']

In [331]:
external_data_service.get_database_schemas("postgresql", test_db, test_user, test_host, test_port, test_password)

[{'schema_name': 'aidm', 'db_instance': 'db_data_catalog'},
 {'schema_name': 'aidm_webview', 'db_instance': 'db_data_catalog'},
 {'schema_name': 'authentication', 'db_instance': 'db_data_catalog'},
 {'schema_name': 'dashboard', 'db_instance': 'db_data_catalog'},
 {'schema_name': 'datamart', 'db_instance': 'db_data_catalog'},
 {'schema_name': 'datamart_layer_1', 'db_instance': 'db_data_catalog'},
 {'schema_name': 'datamart_layer_2', 'db_instance': 'db_data_catalog'},
 {'schema_name': 'datamart_metadata', 'db_instance': 'db_data_catalog'},
 {'schema_name': 'hrsv_raw_dataset', 'db_instance': 'db_data_catalog'},
 {'schema_name': 'mariadb_data_raw_dataset', 'db_instance': 'db_data_catalog'},
 {'schema_name': 'mysql_classicmodels_raw_dataset',
  'db_instance': 'db_data_catalog'},
 {'schema_name': 'mysql_sharing_vision_raw_dataset',
  'db_instance': 'db_data_catalog'},
 {'schema_name': 'notification', 'db_instance': 'db_data_catalog'},
 {'schema_name': 'pg_toast', 'db_instance': 'db_data_cata

In [340]:
data_service.get_id_database_by_instance("db_data_catalog")

5

In [463]:
data_service.collect_and_add_schemas("postgresql", test_db, test_user, test_host, test_port, test_password)

Schemas added successfully for database ID 5.


In [611]:
importlib.reload(metrics)
evaluator = metrics.DataQualityEvaluator(engine)
data_service = metrics.DataService(engine)
external_data_service = metrics.ExternalDatabaseService()

In [546]:
external_data_service.get_database_table("postgresql", test_db, test_user, test_host, test_port, test_password)[0]

{'table_name': 'db_info',
 'total_columns': 11,
 'total_rows': 1,
 'table_size': 49152,
 'schema_name': 'aidm',
 'database_name': 'db_data_catalog'}

In [550]:
data_service.get_id_schema_by_name("hrsv_raw_dataset")

9

In [563]:
data_service.get_id_agent_from_database(5)

3

In [593]:
data_service.get_agent_from_id_agent(3)

'Sharing Vision Indonesia'

In [595]:
data_service.get_database_from_id_database(5)

'db_data_catalog'

In [613]:
data_service.collect_and_add_tables("postgresql", test_db, test_user, test_host, test_port, test_password)

In [10]:
table_name = {"target_table": ["customer_info", "product_info", "office_info", "transaction_monthly"]}
metadata_table = {"table": ["customer_info", "product_info", "office_info"], "label":["Private", "Public", "Public"]}

In [30]:
def security_label(table_name, metadata_table):
    label = []
    for table in table_name["target_table"]:
        label.append(table in metadata_table["table"])
    result = {"table":table_name["target_table"], "label":label}
    print(result)
    
security_label(table_name, metadata_table)

{'table': ['customer_info', 'product_info', 'office_info', 'transaction_monthly'], 'label': [True, True, True, False]}
